## PiRacer 학습코드

210623

##### 파일 경로에 한글이 없어야한다!

- 1) 라즈베리파이에서 수집한 데이터를 zip으로 압축하여 data.zip을 만든다

- 2) 동키카 코드다운 및 설치

- 3) 기존 TF를 삭제하고 TF 1.13.1을 설치

- 4) GPU 확인

- 5) 데이터 업로드

- 6) 데이터 학습!

- 7) 학습모델 colab에서 노트북 다운로드 폴더로 저장

- 8) 다운받은 모델을 라즈베리파이로 전송

- 9) 자율주행 실행!


[ref](https://github.com/robertchoi/self-driving-car-using-google-colab)

### 1) 라즈베리 파이에서 데이터 준비

'data' 폴더 안에 'tub_31_20-05-16'과 같이 폴더들이 있고, 이 폴더 안에 jpg와 json 파일들이 들어있다.  
이 때 data 폴더를 압축하여 data.zip을 만든다

### 2) 동키카 코드다운 및 설치

In [1]:
# donkeycar github 다운
!git clone https://github.com/airobotlab/donkeycar
%cd donkeycar
!git checkout master
!pip install -e .[pc]
!donkey createcar --path /content/d2

C:\Users\go\Desktop\test\donkeycar


Cloning into 'donkeycar'...


Branch 'master' set up to track remote branch 'master' from 'origin'.


Switched to a new branch 'master'


Obtaining file:///C:/Users/go/Desktop/test/donkeycar
  Using cached numpy-1.19.5-cp36-cp36m-win_amd64.whl (13.2 MB)
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13709 sha256=eb9fc3a943ee178ab95fc0fac3108b755fd35c69b214c48fedef66109d575664
  Stored in directory: c:\users\go\appdata\local\pip\cache\wheels\3f\2a\fa\4d7a888e69774d5e6e855d190a8a51b357d77cc05eb1c097c9
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110731 sha256=62b4a7d9e495f96f03afa029abb579c332941f5a49e336cfe6d79e7b975883d6
  Stored in directory: c:\users\go\appdata\local\pip\cache\wheels\be\dc\17\8b4d5a63bcd05dc44db7da57e193372ccd333617293f9deebe
  Created wheel for proglog: filename=proglog-0.1.9-py3-none-any.whl size=6151 sha256=3eeb69f85cbc0885ddfd38c6690d692ac0cc6ed455a46b0c24e00cf99964ea9b
  Stored in directory: c:\users\go\appdata\local\pip\cache\wheels\e7\11\a0\7e65f734d33043735a557b1244569cca327353db9068158076
  Created wheel for paho-mqtt: filename=paho_mqt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moviepy 1.0.3 requires numpy>=1.17.3; python_version != "2.7", but you have numpy 1.17.0 which is incompatible.


### 3) 기존 TF를 삭제하고 TF 1.13.1을 설치

In [2]:
!pip uninstall -y tensorflow  # 기존 설치된 tf 삭제

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1


In [3]:
!pip install tensorflow-gpu==1.13.1  # 2.0.0 버전 설치 

     |████████████████████████████████| 345.0MB 41kB/s 
     |████████████████████████████████| 368kB 48.7MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 3.2MB 45.5MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1


### 4) GPU 확인

"Found GPU at: / device: GPU: 0" 가 뜬다면 GPU 사용 가능

In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Found GPU at: /device:GPU:0


### 5) 데이터 업로드

'data' 폴더 안에 'tub_31_20-05-16'과 같이 폴더들이 있고, 이 폴더 안에 jpg와 json 파일들이 들어있다.  
이 때 data 폴더를 압축하여 data.zip을 만든다

data.zip을 아래 코드로 업로드한다.  
업로드 되면 /content/data.zip를 /content/d2/data/ 경로로 이동 후 압축을 해제한다.  
최종적으로 /content/d2/data/ 폴더 아래에 tub~ 폴더가 생성되고, 이 안에 jpg와 json 파일들이 있다.  
2000장 이상이 필요하다

#Prepare the data: upload data.zip and unzip data
When the following code is executed, a file upload form appears. Upload data.zip.
The data.zip file is the folder found from your raspberry pi. Copy the data to your pc/mac and then zip it as data.zip
data.zip is uploaded under / content. Move to /content/d2/data using the mv command and unzip it.

<img src="https://i1.wp.com/techwithsach.com/wp-content/uploads/2019/06/Screen-Shot-2019-06-29-at-7.43.15-PM.png?fit=1024%2C658&amp;ssl=1" alt="Train an autopilot using Colab" class="wp-image-872"/>

In [14]:
# 압축된 데이터 파일 업로드
import os
from google.colab import files

if(os.path.exists("/content/data.zip")):
    os.remove("/content/data.zip")
if(os.path.exists("/content/d2/data/data.zip")):
    os.remove("/content/d2/data/data.zip")

uploaded = files.upload()  # data.zip 업로드

WORK_FOLDER = "/content/d2/data/"
if(os.path.exists(WORK_FOLDER) == False):
    os.makedirs(WORK_FOLDER)

!mv /content/data.zip /content/d2/data/
%cd /content/d2/data/
!unzip -o data.zip  # 압축 해제

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: tub_31_20-05-16/82_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/830_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/831_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/832_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/833_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/834_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/835_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/836_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/837_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/838_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/839_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/83_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/840_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/841_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/842_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/843_cam-image_array_.jpg  
  inflating: tub_31_20-05-16/844_cam-image_array_.jpg  
  inflating: t

### 6) 데이터 학습!

manage.py 함수를 이용하여 데이터를 학습한다.  
학습이 완료되면 /content/d2/models/mypilot.h5 경로에 저장된다.

In [15]:
!python /content/d2/manage.py train --model /content/d2/models/mypilot.h5

using donkey v3.1.0 ...
loading config file: /content/d2/config.py
loading personal config over-rides

config loaded
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

### 7) 학습모델 colab에서 노트북 다운로드 폴더로 저장

colab의 /content/d2/models/mypilot.h5 에 저장된 모델을 노트북의 다운로드 폴더로 자동 저장

In [2]:
%cd /content/d2/models
from google.colab import files
files.download('mypilot.h5')

[WinError 3] 지정된 경로를 찾을 수 없습니다: '/content/d2/models'
C:\Users\go\Desktop\test\donkeycar


ModuleNotFoundError: No module named 'google.colab'

### 8) 다운받은 모델을 라즈베리파이로 전송

###### usb, scp 등으로 라즈베리파이의 ~/mycar/models/ 로 보낸다.  
scp <<노트북의 mypilot.h5 경로>>/mypilot.h5 pi@<<라즈베리파이 IP>>:~/mycar/models/

### 9) 자율주행 실행!

라즈베리파이에서 
cd ~/mycar
python manage.py drive --model ~/mycar/models/mypilot.h5

노트북 웹에서 
http://<raspberrypi_ip_address>:8887
